In [1]:
# import the libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
import csv

## I- login to Linkedin

In [2]:
# login to Linkedin

# open Chrome and access Linkedin login site
driver = webdriver.Chrome()
sleep(5)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('- Finish initializing a driver')
sleep(5)

- Finish initializing a driver


In [3]:
# import username and password
credential = open('login_credentials.txt')
line = credential.readlines()
username = line[0].strip()
password = line[1]
print('- Finish importing the login credentials')
sleep(2)

- Finish importing the login credentials


In [4]:
# key in login credentials
email_field = driver.find_element(by=By.ID, value='username')
email_field.send_keys(username)
print('- Finish keying in email')
sleep(3)

password_field = driver.find_element(by=By.NAME, value='session_password')
password_field.send_keys(password)
print('- Finish keying in password')
sleep(5)

- Finish keying in email
- Finish keying in password


In [5]:
# click the Login button
signin_field = driver.find_element(by=By.XPATH, value='//*[@id="organic-div"]/form/div[3]/button')
signin_field.click()
sleep(3)

print('- Finish: Login to Linkedin')

- Finish: Login to Linkedin


## II- search for the profiles

In [6]:
# locate the search bar element
search_field = driver.find_element(by=By.XPATH, value='//*[@id="global-nav-typeahead"]/input')

# input the search query to the search bar
search_query = 'business student people'
search_field.send_keys(search_query)
sleep(2)

# search
search_field.send_keys(Keys.RETURN)
sleep(5)
print('- Finish: Search for profiles')

- Finish: Search for profiles


In [7]:
# click the people button
people_field = driver.find_element(by=By.XPATH, value='//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')
people_field.click()
sleep(3)

print('- Finish: select peole')

- Finish: select peole


### filter by business students

In [8]:
# click the All Filters button
all_filters_field = driver.find_element(by=By.XPATH, value='//*[@id="search-reusables__filters-bar"]/div/div/button')
all_filters_field.click()
sleep(3)

print('- Finish: select all filters button')


# search by the keyword 'title'

# locate the search bar element
title_field = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/div/div[2]/ul/li[12]/ul/li[3]/label/input')
title_field.send_keys('business student')
sleep(2)

# locate the 'show results' button
results_field = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div/div/div[3]/div/button[2]/span')
results_field.click()
sleep(3)
print('- Finish: filter by business student profiles')

- Finish: select all filters button
- Finish: filter by business student profiles


## III- scrape the urls of the business student profiles

In [9]:
# step to ignore the profiles listed as 'LinkedIn Member'
x = 'https://www.linkedin.com/search/results/people/headless?title=business%20student&origin=FACETED_SEARCH&keywords=business%20student%20people'

In [10]:
# function to extract the URLs of one page

def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link scale-down')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = profile_ID
        if profile_URL not in all_profile_URL and profile_URL != x:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

In [11]:
# navigate through many pages, and extract the profile URLs of each page

num_page = 32 # number of pages to scrape
URLs_all_page = []
for page in range(num_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    
    while True:
        try:
            next_button = driver.find_element(by=By.CLASS_NAME, value="artdeco-pagination__button--next")
            driver.execute_script("arguments[0].click();", next_button)
            
        except:
            continue
        
        else:
            URLs_all_page = URLs_all_page + URLs_one_page
            sleep(2)
            break

print('- Finish: Scrape the URLs')

- Finish: Scrape the URLs


In [21]:
print(f'Scraped the URLs of {len(URLs_all_page)} student profiles.')

Scraped the URLs of 114 student profiles.


## IV- scrape the data of the Linkedin profiles, and write the data to a .CSV file

In [ ]:
with open('business_students.csv', 'w',  newline = '') as file_output:
    headers = ['name', 'title', 'location', 'connections', 'education', 'school', 'language', 'about']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
            sleep(15)
            driver.get(linkedin_URL)
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            print('- Accessing profile: ', linkedin_URL)                

            try:
                    sleep(15)
                    page_source = BeautifulSoup(driver.page_source, "html.parser")
                    soup = BeautifulSoup(driver.page_source, 'lxml')



                    # general info
                    info_div = page_source.find('div',{'class':'ph5 pb5'})

                    name = info_div.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip()
                    print(name)

                    title = info_div.find('div', class_='text-body-medium break-words').get_text().strip()
                    print(title)

                    loc = info_div.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip()
                    print(loc)

                    connections = info_div.find('span', class_='t-bold').get_text().strip()
                    print(connections)


                    # education

                    edu = soup.find("div", {"id": "education"})
                    education = edu.parent.find('span', class_='t-14 t-normal').get_text().strip()
                    education = education[:int(len(education)/2)]

                    school = edu.parent.find('span', class_='mr1 hoverable-link-text t-bold').get_text().strip()
                    school = school[:int(len(school)/2)]

                    print(education)
                    print(school)


                    # language


                    language = soup.find("div", {"id": "languages"})

                    lang1=[]
                    for lang in language.parent.find_all('span', class_='mr1 t-bold'):
                        x = lang.get_text().strip()
                        x = x[:int(len(x)/2)]
                        lang1.append(x)

                    print(lang1)


                    # about


                    about = soup.find("div", {"id": "about"})

                    about_list=[]
                    for x in about.parent.find_all('span', class_='visually-hidden'):
                        about_list.append(x.get_text().strip())

                    print(about_list[1])

                    sleep(5)
                    writer.writerow({headers[0]:name, 
                                    headers[1]:title, 
                                    headers[2]:loc, 
                                    headers[3]:connections, 
                                    headers[4]:education, 
                                    headers[5]:school, 
                                    headers[6]:lang1, 
                                    headers[7]:about_list[1]})
                    print('\n')



            except:
                    print('error/exception')
                    print('\n')


print('- Completed!')

- Accessing profile:  https://www.linkedin.com/in/in%C3%A9s-ebanga?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABr2K_ABNwJ75zLH14miPWRzRcvaX4V5lb8
Inés Ebanga Etoundi
Student at emlyon business school
Écully, Auvergne-Rhône-Alpes, France
500+
Master's degree in management , Operations Management and Supervision
emlyon business school
['English', 'French', 'Spanish']
Student in first year of master's degree in management and Generalist engineer-major in Industrial engineering- with an interest in professions at the heart of operational strategy development.


- Accessing profile:  https://www.linkedin.com/in/sami-lakis-9363aa157?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACWchroBitGwkimNkrthbDfKkOjc62AkjoY
Sami Lakis
Business student MBA with a Major in Sports Management
Barcelona, Catalonia, Spain
500+
Master of Business Administration - MBA with a Major in Sports Management, Business Administration and Management, General
EU Business School
['Arabic', 'English', 'French']
I am 

Aashish Aswani
Student at The George Washington University - School of Business
, Bachelor of Science in Management Information Systems
Washington, District of Columbia, United States
163
Bachelor of Science - BS, Management Information Systems, General
The George Washington University
['French', 'Hindi', 'Sindhi']
I am currently an international student studying in the George Washington School of Business to pursue a degree in Management Information Systems and International Business. Throughout my time here, I hope to gain valuable experiences and insights in order to propel me towards my desired job. Currently, my interests include business on an international level, coding, data analysis, and management strategies. I am interested in meeting people who share a similar background and would like to explore more ways to expand on my current knowledge as well as achieve new skills through internships and experiences. As a student from various cultural backgrounds, I enjoy interacting w



- Accessing profile:  https://www.linkedin.com/in/mridula-sthapit-9145ab1aa?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADDalWABwhakWtGqfk65nuYKUth4kF0Nipc
Mridula Sthapit
Business Student at People's Campus, Tribhuvan University
Bāgmatī, Nepal
171
Bachelor of Business Administration - BBA, Business Administration and Management, General
Tribhuvan University
error/exception


- Accessing profile:  https://www.linkedin.com/in/paraskulkarni?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABJLCcIBLvFcMz9tog1T66xXAoKNDFiVvDQ
Paras Kulkarni
MBA Student at Cologne Business School
Cologne, North Rhine-Westphalia, Germany
500+
Master of Business Administration - MBA, Business Administration and Management, General
CBS International Business School
error/exception


- Accessing profile:  https://www.linkedin.com/in/mulaika-shaikh-4a4011215?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADZI6nwBZtmZSN_2z1yrgb_L5Mf18h1Fogs
Mulaika Shaikh
Business Student at Westford International College
Duba

Ryan Neary
Student at Fordham's Gabelli School of Business
Bronx, New York, United States
266
Bachelor of Business Administration - BBA, Finance, General
Fordham Gabelli School of Business
error/exception


- Accessing profile:  https://www.linkedin.com/in/ujwala-s-u-3a2aa415a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACY_nyoBHa8oSTarRzb10Ai9bIrjl3aGWR4
error/exception


- Accessing profile:  https://www.linkedin.com/in/melisa-gale-25b6bb225?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADiF3qcBoWGEqvsqvlD-1fe4s0FEMXI-RL8
Melisa Gale
Business Student at Niagara College
Welland, Ontario, Canada
12
Associate's degree, Accounting and Business/Management
Niagara College
error/exception


- Accessing profile:  https://www.linkedin.com/in/humza-butt-9a07bb1b4?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADHz3sgBbaPQsCHiO8IUP3lbYRrICBfBFM4
Humza Butt
Business Student at The Manchester Metropolitan University
Manchester, England, United Kingdom
422
Bachelor of Arts - BA, business manag

In [15]:
try:
    name=1/0
except:
    name=0
print(name)

0


In [17]:
with open('business_students.csv', 'w',  newline = '') as file_output:
    headers = ['name', 'title', 'location', 'connections', 'education', 'school', 'language', 'about']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
            driver.get(linkedin_URL)
            sleep(15)
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            print('- Accessing profile: ', linkedin_URL)                

            try:
                    sleep(7)
                    page_source = BeautifulSoup(driver.page_source, "html.parser")
                    soup = BeautifulSoup(driver.page_source, 'lxml')



                    # general info
                    info_div = page_source.find('div',{'class':'ph5 pb5'})

                    name = info_div.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip()
                    print(name)

                    title = info_div.find('div', class_='text-body-medium break-words').get_text().strip()
                    print(title)

                    loc = info_div.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip()
                    print(loc)

                    connections = info_div.find('span', class_='t-bold').get_text().strip()
                    print(connections)


                    # education

                    edu = soup.find("div", {"id": "education"})
                    education = edu.parent.find('span', class_='t-14 t-normal').get_text().strip()
                    education = education[:int(len(education)/2)]

                    school = edu.parent.find('span', class_='mr1 hoverable-link-text t-bold').get_text().strip()
                    school = school[:int(len(school)/2)]

                    print(education)
                    print(school)


                    # language

                    try:
                        language = soup.find("div", {"id": "languages"})

                        lang1=[]
                        for lang in language.parent.find_all('span', class_='mr1 t-bold'):
                            x = lang.get_text().strip()
                            x = x[:int(len(x)/2)]
                            lang1.append(x)

                        print(lang1)
                    
                    except:
                        lang1= 'Nan'

                    # about

                    try:
                        about = soup.find("div", {"id": "about"})

                        about_list=[]
                        for x in about.parent.find_all('span', class_='visually-hidden'):
                            about_list.append(x.get_text().strip())

                        print(about_list[1])
                    
                    except:
                        about_list=['Nan','Nan']
                    

                    sleep(5)
                    writer.writerow({headers[0]:name, 
                                    headers[1]:title, 
                                    headers[2]:loc, 
                                    headers[3]:connections, 
                                    headers[4]:education, 
                                    headers[5]:school, 
                                    headers[6]:lang1, 
                                    headers[7]:about_list[1]})
                    print('\n')



            except:
                    print('error/exception')
                    print('\n')


print('- Completed!')

- Accessing profile:  https://www.linkedin.com/in/in%C3%A9s-ebanga?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABr2K_ABNwJ75zLH14miPWRzRcvaX4V5lb8
Inés Ebanga Etoundi
Student at emlyon business school
Écully, Auvergne-Rhône-Alpes, France
500+
Master's degree in management , Operations Management and Supervision
emlyon business school
['English', 'French', 'Spanish']
Student in first year of master's degree in management and Generalist engineer-major in Industrial engineering- with an interest in professions at the heart of operational strategy development.


- Accessing profile:  https://www.linkedin.com/in/sami-lakis-9363aa157?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACWchroBitGwkimNkrthbDfKkOjc62AkjoY
Sami Lakis
Business student MBA with a Major in Sports Management
Barcelona, Catalonia, Spain
500+
Master of Business Administration - MBA with a Major in Sports Management, Business Administration and Management, General
EU Business School
['Arabic', 'English', 'French']
I am 

Austin K.
Business Student at The University of Tennessee at Chattanooga
Chattanooga, Tennessee, United States
54
Associate's degree, Entrepreneurship/Entrepreneurial Studies
Chattanooga State Community College
I am a student/aspiring entrepreneur who enjoys connecting the dots: be it ideas from different disciplines, people from different teams, or applications from different industries. I am practicing strong business skills and building an academic background in entrepreneurship, marketing, and business development.


My passion lies in learning business in all capacities, from international marketing to small business ventures. I am able to jump across verticals to make my dreams and ideas work for me.


I will be graduating in May 2023 and I'm interested in brand marketing and startup positions like product director. Please feel free to get in touch with me via email at kalick52598@gmail.com.


- Accessing profile:  https://www.linkedin.com/in/shambhavi-mishra-6b3356168?miniProfil



- Accessing profile:  https://www.linkedin.com/in/emily-parnell-a380b8173?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACkgpswBXOQOXcZ9l58oJM73sr1mXWjQJs8
Emily Parnell
Business Student at University of East Anglia
Greater Bournemouth Area
51
Bachelor's degree, International Business
University of East Anglia
['French']
I am a reliable, highly motivated, ambitious person who communicates well with people of all ages. I am curious and quick to learn new things.  I have a friendly, approachable manner and I am confident at working independently or as part of a team. I am always committed to doing everything to the best of my ability while learning how to do it even better.  I really enjoy working with customers and seeing their satisfaction in the service I provide.  I have work experience in many sectors through the various part-time jobs I have held.  I am looking for new career opportunities and to network with others in business.


- Accessing profile:  https://www.linkedin.com/i



- Accessing profile:  https://www.linkedin.com/in/mulaika-shaikh-4a4011215?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADZI6nwBZtmZSN_2z1yrgb_L5Mf18h1Fogs
Mulaika Shaikh
Business Student at Westford International College
Dubai, Dubai, United Arab Emirates
90
BBA hons in international business, Business Administration and Management, General
Westford International College
An enthusiastic young mind looking forward to applying all her experiences into any industry which will provide more experience and allow to use skills gained in the best way possible towards the enhancement of the industry and self-profile


- Accessing profile:  https://www.linkedin.com/in/noura-almuneer-60104b1b9?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADK64aEBRmVBcCSwmS7rYspuGIlxsq2yS2o
Noura Almuneer
Business Student at Kuwait University
Al Asimah, Kuwait
20
Bachelor's degree, Marketing
Kuwait University
Hi, I am a Digital Marketer, and an artist who likes to be creative and unique in every possible wa



- Accessing profile:  https://www.linkedin.com/in/ujwala-s-u-3a2aa415a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACY_nyoBHa8oSTarRzb10Ai9bIrjl3aGWR4
error/exception


- Accessing profile:  https://www.linkedin.com/in/melisa-gale-25b6bb225?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADiF3qcBoWGEqvsqvlD-1fe4s0FEMXI-RL8
Melisa Gale
Business Student at Niagara College
Welland, Ontario, Canada
12
Associate's degree, Accounting and Business/Management
Niagara College
Always been detailed with my numbers and highly organized. Would love to use my skills, current and new ones being learned through schooling to help Businesses thrive, grow and continue on their paths of success. 
I like to stay on top of new trends in the accounting and business world to further myself, my family and society.


- Accessing profile:  https://www.linkedin.com/in/humza-butt-9a07bb1b4?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADHz3sgBbaPQsCHiO8IUP3lbYRrICBfBFM4
Humza Butt
Business Student at The Manc

...



- Accessing profile:  https://www.linkedin.com/in/olivia-huss?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC5XIdwBkd3Zv0HrpcH_v9B6XyDm6VTfrdQ
Olivia Huss
Business Management Student
New York City Metropolitan Area
498
Bachelor's degree, Business Administration, Management and Operations, Spanish Minor
Loyola University Maryland
['Spanish']
Fourth year business management student, aspiring to work collaboratively and manage others. My passion for teamwork has led me to concentrate my studies at Loyola University Maryland to business management. I have always found that working together and problem solving with a small group of people challenges my thinking and opens my eyes through the perspectives of others. 

I would love to be able to, and look forward to, immersing myself in the management culture and experience what working professionally on a team is like. I continue to challenge myself in the classroom and connect with my peers as well as the amazing business professors at L



- Accessing profile:  https://www.linkedin.com/in/jasmine-phillips-uni824?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACCEzcQB1aai0ihbeFyG0dSn-fnEjRRgHW4
Jasmine Phillips
Jasmine Phillips - Business Student @ MRU
Calgary, Alberta, Canada
500+
Bachelor of Business Administration - BBA
Mount Royal University
Currently attending Mount Royal University as a student for a Bachelor of Business Administration and Freelance Digital Marketer. 

I love the experience I have gained in my career through many different industries such as agriculture, oil & gas, public service and architecture. I am well-rounded, versatile and have acquired many skills which I utilize to contribute to the overall growth of a business.   

I love meeting new people, developing new connections, and problem solving as a team.


- Accessing profile:  https://www.linkedin.com/in/vyvytrant?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC_xvkwBwELU6aDsXflcTcaR8tcsPD--7vw
Vyvy Tran
Business Management Student at Texas



- Accessing profile:  https://www.linkedin.com/in/karen-kanisha?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACpp8ysBfXbnWXjt5sN5WUeV9JsdpPGYO4g
Karen kanisha
Fashion Business Management Student at University of Westminster
Lewisham, England, United Kingdom
61
Fashion Design, Fashion/Apparel Design
Limkokwing University of Creative Technology
Hi there, I am a Fashion Design Graduate from Limkokwing University  Malaysia, currently pursuing my MA in Fashion Business Management with University of Westminster. I intend to see a future of a 0 Waste  fashion industry and educating the upcoming generation regarding sustainability, creativity and towards a sustainable future ahead.I am looking for opportunities to learn more and share more, lets connect here on LinkedIn :)
For A Better Future Ahead!


- Accessing profile:  https://www.linkedin.com/in/bart-tonna-4855803?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAACeNqUB_A1F49Jq5WmTudVTnS8Vq0WaQFw
Bart Tonna
Student at University of Syd



- Accessing profile:  https://www.linkedin.com/in/shreyaaggarwal01?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADKBzowBdOP-dcHmRCLit2ZuNCIsUtwOEo8
error/exception


- Accessing profile:  https://www.linkedin.com/in/shannon-b-414691135?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACEGuhABxadV_e7dY1PAiS6l62-trbiX-QQ
Shannon B.
Accounting Student at The Dhillon School of Business
Calgary, Alberta, Canada
151
Bachelor of Management, Accounting
The University of Lethbridge
I am a recent graduate from the Dhillon School of Business, passionate on pursuing a career in accounting with intentions of following the CPA path. Being well versed in customer service experience along with my undergraduate degree in Business Management allows for much flexibility in cross functional roles. 

In my free time, you can catch me at the dog park with my pup or teaching her new tricks.


- Accessing profile:  https://www.linkedin.com/in/trang-mai-482b62175?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AAC

error/exception


- Completed!
